<a href="https://colab.research.google.com/github/yunnsss/Fog_Forecast/blob/main/ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import SparseCategoricalCrossentropy

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, mean_absolute_error
import seaborn as sns

import imblearn
from imblearn.over_sampling import SMOTE

from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
# Load data
train = pd.read_csv('/content/drive/MyDrive/날씨마루 공모전/train_FE.csv')
test = pd.read_csv('/content/drive/MyDrive/날씨마루 공모전/test_FE.csv')

In [ ]:
# 필요없는 열 제거
train = train.drop(labels=['sin_week', 'cos_week',
                                'sin_year', 'cos_year'
                                ], axis = 1)

test = test.drop(labels=['sin_week', 'cos_week',
                                'sin_year', 'cos_year'
                                ], axis = 1)

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

train.isnull().sum()

year          0
month         0
day           0
time          0
minute        0
stn_id        0
ws10_deg      0
ws10_ms       0
ta            0
re            0
hm            0
sun10         0
ts            0
vis1          0
class         0
datetime      0
season        0
sin_day       0
cos_day       0
ta_tb_diff    0
sin_month     0
cos_month     0
st            0
ws10_new      0
dtype: int64

In [ ]:
test.isnull().sum()

year          0
month         0
day           0
time          0
minute        0
stn_id        0
ws10_deg      0
ws10_ms       0
ta            0
re            0
hm            0
sun10         0
ts            0
class         0
datetime      0
season        0
sin_day       0
cos_day       0
ta_tb_diff    0
sin_month     0
cos_month     0
st            0
ws10_new      0
dtype: int64

In [ ]:
# stn_id의 첫 번째 글자 추출
train['group'] = train['stn_id'].str[0]
test['group'] = test['stn_id'].str[0]

# 각 글자별로 데이터프레임 분리
stn_A = train[train['group'] == 'A']
stn_B = train[train['group'] == 'B']
stn_C = train[train['group'] == 'C']
stn_D = train[train['group'] == 'D']
stn_E = train[train['group'] == 'E']

# 테스트 데이터프레임 분리
test_stn_A = test[test['group'] == 'A']
test_stn_B = test[test['group'] == 'B']
test_stn_C = test[test['group'] == 'C']
test_stn_D = test[test['group'] == 'D']
test_stn_E = test[test['group'] == 'E']

In [ ]:
# Function to create and train the model for each stn_id
def train_and_evaluate_stn(stn_data, stn_name):
    X = stn_data.drop(labels=['year','month','day','time','minute', 'class', 'season', 'stn_id', 'group', 'datetime', 'vis1'], axis=1)
    y = stn_data['class']

    # Split the data into 80% training and 20% testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

    # SMOTE oversampling
    smote = SMOTE()
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Initializing the ANN
    classifier = Sequential()

    # Adding the input layer and the first hidden layer
    classifier.add(Dense(units=1024, kernel_initializer='uniform', activation='swish', input_dim=X_train.shape[1]))
    classifier.add(Dense(units=512, kernel_initializer='uniform', activation='swish'))
    classifier.add(Dense(units=256, kernel_initializer='uniform', activation='swish'))
    classifier.add(Dense(units=128, kernel_initializer='uniform', activation='swish'))
    classifier.add(Dense(units=64, kernel_initializer='uniform', activation='swish'))
    classifier.add(Dense(units=16, kernel_initializer='uniform', activation='swish'))
    # Adding the output layer
    classifier.add(Dense(units=4, kernel_initializer='uniform', activation='softmax'))

    # Compiling the ANN
    classifier.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

    # Fitting the ANN to the Training set
    classifier.fit(X_train_resampled, y_train_resampled - 1, batch_size=20, epochs=2)  # Subtract 1 to make labels 0-3

    # Predicting the Test set results
    y_pred = classifier.predict(X_test)

    # Converting predictions to class labels
    y_pred_classes = y_pred.argmax(axis=-1) + 1  # Add 1 to convert labels back to 1-4

    # Calculate accuracy and F1 score
    f1 = f1_score(y_test, y_pred_classes, average='weighted')
    mae = mean_absolute_error(y_test, y_pred_classes)
    accuracy = accuracy_score(y_test, y_pred_classes)

    print(f"{stn_name} ANN MAE: {mae}")
    print(f"{stn_name} ANN 정확도: {accuracy}")
    print(f"{stn_name} ANN F1 점수: {f1}")

    return classifier, y_test, y_pred_classes

In [ ]:
# 각 데이터프레임에 대한 ANN 모델 학습 및 평가
#ANN_stn_A, f1_stn_A, mae_stn_A = train_and_evaluate_stn(stn_A, '_A_')
#ANN_stn_B, f1_stn_B, mae_stn_B = train_and_evaluate_stn(stn_B, '_B_')
#ANN_stn_C, f1_stn_C, mae_stn_C = train_and_evaluate_stn(stn_C, '_C_')
#ANN_stn_D, f1_stn_D, mae_stn_D = train_and_evaluate_stn(stn_D, '_D_')
#ANN_stn_E, f1_stn_E, mae_stn_E = train_and_evaluate_stn(stn_E, '_E_')

# 각 데이터프레임에 대해 함수 호출 및 결과 저장
results = {}
ANN_A, y_test_A, y_pred_A = train_and_evaluate_stn(stn_A, 'A')
results['A'] = (y_test_A, y_pred_A)
ANN_B, y_test_B, y_pred_B = train_and_evaluate_stn(stn_B, 'B')
results['B'] = (y_test_B, y_pred_B)
ANN_C, y_test_C, y_pred_C = train_and_evaluate_stn(stn_C, 'C')
results['C'] = (y_test_C, y_pred_C)
ANN_D, y_test_D, y_pred_D = train_and_evaluate_stn(stn_D, 'D')
results['D'] = (y_test_D, y_pred_D)
ANN_E, y_test_E, y_pred_E = train_and_evaluate_stn(stn_E, 'E')
results['E'] = (y_test_E, y_pred_E)

Epoch 1/2
199064/199064 [==============================] - 2487s 12ms/step - loss: 0.2941 - accuracy: 0.8869
Epoch 2/2
7817/7817 [==============================] - 32s 4ms/step
A ANN MAE: 0.05776565885956004
A ANN 정확도: 0.9564278688917853
A ANN F1 점수: 0.9738648475256357
Epoch 1/2
98225/98225 [==============================] - 1225s 12ms/step - loss: 0.6257 - accuracy: 0.7284
Epoch 2/2
3926/3926 [==============================] - 15s 4ms/step
B ANN MAE: 0.11803062028773198
B ANN 정확도: 0.9269046121509836
B ANN F1 점수: 0.9494142685916205
Epoch 1/2
50088/50088 [==============================] - 617s 12ms/step - loss: 0.3872 - accuracy: 0.8429
Epoch 2/2
1968/1968 [==============================] - 9s 4ms/step
C ANN MAE: 0.07626943663537745
C ANN 정확도: 0.9568145359825924
C ANN F1 점수: 0.9738909688808953


In [ ]:
# 전체 결과 출력
for key, (y_test, y_pred) in results.items():
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    mae = mean_absolute_error(y_test, y_pred)
    print(f"{key} - Accuracy: {accuracy}, F1 Score: {f1}, MAE: {mae}")

A - Accuracy: 0.9564278688917853, F1 Score: 0.9738648475256357, MAE: 0.05776565885956004
B - Accuracy: 0.9269046121509836, F1 Score: 0.9494142685916205, MAE: 0.11803062028773198
C - Accuracy: 0.9568145359825924, F1 Score: 0.9738909688808953, MAE: 0.07626943663537745


In [ ]:
# 결과 데이터 생성
results = {
    'Station': ['A', 'B', 'C', 'D', 'E'],
   #'MAE': [mae_stn_A, mae_stn_B, mae_stn_C, mae_stn_D, mae_stn_E],
    # 'Accuracy': [f1_stn_A, f1_stn_B, f1_stn_C, f1_stn_D, f1_stn_E],
    'F1-score': [f1_stn_A, f1_stn_B, f1_stn_C, f1_stn_D, f1_stn_E]
}

result_df = pd.DataFrame(results)

# 데이터프레임 출력
print(result_df)

NameError: name 'mae_stn_A' is not defined

In [ ]:
# MAE 시각화
plt.figure(figsize=(5, 3))
stations = ['A', 'B', 'C', 'D', 'E']
mae_values = [mae_stn_A, mae_stn_B, mae_stn_C]

plt.bar(stations, mae_values)
plt.xlabel('Station')
plt.ylabel('MAE')
plt.title('Mean Absolute Error (MAE) for Each Station')
plt.show()

In [ ]:
# F1-score 시각화 (레이더 차트)
plt.figure(figsize=(8, 8))
f1_scores = [f1_stn_A, f1_stn_B, f1_stn_C, f1_stn_D, f1_stn_E]

# 레이더 차트 그리기
radar_data = pd.DataFrame({'Station': stations, 'F1-score': f1_scores})
radar_data = radar_data.set_index('Station')

ax = plt.subplot(projection='polar')
ax.plot(np.linspace(0, 2 * np.pi, len(f1_scores)), f1_scores, linewidth=2)
ax.fill(np.linspace(0, 2 * np.pi, len(f1_scores)), f1_scores, alpha=0.5)
ax.set_thetagrids(np.degrees(np.linspace(0, 2 * np.pi, len(f1_scores))), labels=stations)
ax.set_title('F1-score for Each Station')
plt.show()

In [ ]:
# 예측 함수
def predict_and_fill(stn_test, model):
    X = stn_test.drop(labels=['year','month','day','time','minute', 'season', 'stn_id', 'group', 'datetime', 'class'], axis=1)
    y_pred = model.predict(X)

    # Assigning predicted classes back to season_df
    stn_test['class'] = y_pred

    return stn_test

In [ ]:
# 각 데이터프레임에 대해 함수 호출
pred_A = predict_and_fill(test_stn_A, ANN_A)
pred_B = predict_and_fill(test_stn_B, ANN_B)
pred_C = predict_and_fill(test_stn_C, ANN_C)
pred_D = predict_and_fill(test_stn_D, ANN_D)
pred_E = predict_and_fill(test_stn_E, ANN_E)

  31/1643 [..............................] - ETA: 5s

<ipython-input-29-6235e2f76afa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stn_test['class'] = y_pred


  11/1643 [..............................] - ETA: 9s 

<ipython-input-29-6235e2f76afa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stn_test['class'] = y_pred


1643/1643 [==============================] - 7s 4ms/step


<ipython-input-29-6235e2f76afa>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stn_test['class'] = y_pred


In [ ]:
# 데이터프레임 합치기
result_ANN = pd.concat([pred_A, pred_B, pred_C, pred_D, pred_E])

In [ ]:
unique_values, counts = np.unique(y_pred_A, return_counts=True)
print(dict(zip(unique_values, counts)))
unique_values, counts = np.unique(y_pred_B, return_counts=True)
print(dict(zip(unique_values, counts)))
unique_values, counts = np.unique(y_pred_C, return_counts=True)
print(dict(zip(unique_values, counts)))
unique_values, counts = np.unique(y_pred_D, return_counts=True)
print(dict(zip(unique_values, counts)))
unique_values, counts = np.unique(y_pred_E, return_counts=True)
print(dict(zip(unique_values, counts)))

{1: 999, 2: 2231, 3: 8204, 4: 238680}
{1: 2030, 2: 3941, 3: 4498, 4: 115134}
{1: 647, 2: 1038, 3: 1171, 4: 60105}


In [ ]:
result_ANN.to_csv('/content/drive/MyDrive/날씨마루 공모전/result_ANN.csv', index=True)